### Creating Stock Data Directory

The directory will be made up of several tables which will have infomation on stock symbols and exchanges that will be later be merged with time series data for each stock. 

The directory scripts will connect to the Financial Model Prep (FMP) API to pull the data.

The API key will be stored in a .gitignore file just to keep it private.

The main table will consist of the following columns:

- symbol
- name
- exchange
- ISIN
- Industry

The data will be stored in as a csv file in the data directory.

In [1]:
import requests
import os
import pandas as pd



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variable
api_key = os.getenv('FMP_API_KEY')

In [3]:
# Testing the API with a search query using LULU as the input symbol

# Define the search query
query = 'LULU'

# Define the endpoint and parameters
endpoint = f'https://financialmodelingprep.com/api/v3/search'
params = {
    'query': query,
    'limit': 10,  # Limit the number of results
    'exchange': 'NASDAQ',  # Optional: specify the exchange
    'apikey': api_key
}

# Make the request
response = requests.get(endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    for company in data:
        print(f"Symbol: {company.get('symbol')}, Name: {company.get('name')}, Exchange: {company.get('exchange', 'N/A')}")
else:
    print(f"Error: {response.status_code}")

Symbol: LULU, Name: Lululemon Athletica Inc., Exchange: N/A
Symbol: LVLU, Name: Lulu's Fashion Lounge Holdings, Inc., Exchange: N/A


In [4]:
# creating a function to get symbols for a specified exchange and return as DataFrame

# Function to get symbols for a specified exchange and return as DataFrame
def get_symbols(exchange):
    # Define the endpoint
    endpoint = f'https://financialmodelingprep.com/api/v3/symbol/{exchange}?apikey={api_key}'

    # Make the request
    response = requests.get(endpoint)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Create a DataFrame from the response data
        df = pd.DataFrame(data)
        return df
    else:
        print(f"Error: {response.status_code}")
        return pd.DataFrame()

In [5]:
# getting the data for the Nasdaq exchange and displaying the first 50 rows

# Example usage
exchanges = ['NYSE', 'NASDAQ']
all_symbols_df = pd.DataFrame()

for exchange in exchanges:
    print(f"Fetching symbols for {exchange} exchange...")
    symbols_df = get_symbols(exchange)
    all_symbols_df = pd.concat([all_symbols_df, symbols_df], ignore_index=True)

Fetching symbols for NYSE exchange...
Fetching symbols for NASDAQ exchange...


In [6]:
# chaning the marketCap column to be displayed in billions

# Display the first 50 rows of the DataFrame
all_symbols_df['marketCap'] = all_symbols_df['marketCap'] / 1e9

all_symbols_df.head(100)


,symbol,name,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,...,exchange,volume,avgVolume,open,previousClose,eps,pe,earningsAnnouncement,sharesOutstanding,timestamp
0,A,"Agilent Technologies, Inc.",139.280,-0.2149,-0.3000,136.7000,139.5950,155.3500,124.1600,40.019044,...,NYSE,1249386.0,1691150.0,138.8100,139.5800,4.420,31.510000,2025-02-25T05:00:00.000+0000,2.873280e+08,1733259602
1,AA,Alcoa Corporation,45.900,0.4376,0.2000,45.5101,46.7500,47.7700,23.8000,11.858495,...,NYSE,2207294.0,5084965.0,46.6500,45.7000,-1.560,-29.420000,2025-01-15T10:59:00.000+0000,2.583550e+08,1733259602
2,AAC,Ares Acquisition Corporation,10.790,0.1857,0.0200,10.7700,10.7900,10.8000,9.0052,0.761820,...,NYSE,599704.0,134092.0,10.7700,10.7700,-0.100,-107.900000,2023-03-31T00:00:00.000+0000,7.060431e+07,1699304402
3,AACT,Ares Acquisition Corporation II,10.930,0.0000,0.0000,10.9101,10.9400,11.0000,10.3700,0.683125,...,NYSE,15136.0,159371.0,10.9400,10.9300,0.440,24.840000,2025-02-26T21:00:00.000+0000,6.250000e+07,1733259602
4,AACT-UN,Ares Acquisition Corporation II,11.090,2.1179,0.2300,11.0900,11.0900,11.3900,10.4300,0.704105,...,NYSE,101.0,1399.0,11.0900,10.8600,NaN,NaN,2025-02-26T21:00:00.000+0000,6.349009e+07,1733236200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,AESC,The AES Corporation,62.610,1.0654,0.6600,62.3000,63.4700,94.3500,47.6899,11.229678,...,NYSE,1175937.0,127212.0,62.8500,61.9500,-0.223,-280.762332,2023-03-31T00:00:00.000+0000,1.793592e+08,1707944488
96,AESI,Atlas Energy Solutions Inc.,24.010,2.4317,0.5700,23.2901,24.1800,24.9300,15.5500,2.646306,...,NYSE,1059017.0,1172003.0,23.5700,23.4400,0.690,34.800000,2025-02-25T05:00:00.000+0000,1.102168e+08,1733259602
97,AET,Aetna Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,NYSE,0.0,0.0,NaN,NaN,NaN,0.000000,2018-09-30T00:00:00.000+0000,0.000000e+00,1561759658
98,AEVA,"Aeva Technologies, Inc.",4.310,-4.4346,-0.2000,4.2101,4.5600,7.2500,2.2010,0.232235,...,NYSE,662767.0,309533.0,4.4900,4.5100,-3.120,-1.380000,2025-02-19T12:00:00.000+0000,5.388280e+07,1733259602


In [7]:
# Save the data to a CSV file

# describe the data
all_symbols_df.describe()


,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,priceAvg50,priceAvg200,volume,avgVolume,open,previousClose,eps,pe,sharesOutstanding,timestamp
count,16477.000000,16419.000000,16422.000000,16466.000000,16466.000000,16351.000000,16350.000000,16889.000000,16741.000000,16741.000000,1.689200e+04,1.674100e+04,16264.000000,16329.000000,14814.000000,11965.000000,1.688900e+04,1.689700e+04
mean,77.369464,1.150230,-0.653224,76.872975,78.449136,102.281373,58.554797,9.366028,85.661044,81.770247,1.124451e+06,7.248057e+05,79.053604,78.706411,24.956690,3.217371,1.416142e+09,1.656870e+09
std,5498.358568,91.534707,79.775695,5482.228253,5587.942589,6035.817634,4174.726844,72.187815,5555.519030,5223.735856,8.963182e+06,3.871020e+06,5621.098188,5603.185671,2074.162159,235.642470,3.582647e+10,3.161423e+08
min,0.000000,-100.000000,-10221.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,-7894.380000,-11000.000000,0.000000e+00,0.000000e+00
25%,6.370000,-0.756850,-0.100000,6.020000,6.780000,9.955000,4.000000,0.017304,5.232500,5.795050,0.000000e+00,0.000000e+00,6.977500,6.910000,-0.050000,-0.660000,3.980386e+06,1.712261e+09
50%,14.240000,0.000000,0.000000,13.780000,14.570000,18.010000,10.950000,0.273901,13.157000,12.918600,1.517000e+04,1.064900e+04,14.600000,14.470000,0.000000,0.680000,2.775110e+07,1.733260e+09
75%,31.000000,0.348600,0.040000,30.686250,31.267500,35.922000,24.450000,2.382686,29.195600,27.994950,2.715492e+05,2.757820e+05,31.340000,31.310001,1.040000,19.730000,1.156738e+08,1.733260e+09
max,705659.000000,9900.000000,94.990000,703355.000000,716919.300000,741971.000000,533700.000000,3667.848870,693299.400000,648919.250000,5.068917e+08,2.554330e+08,716738.000000,715880.000000,235751.050000,5175.000000,2.434297e+12,1.733289e+09


In [8]:
# Save the data to a CSV file in the data directory within the Datasets folder

# Define the file path
file_path = 'data/Datasets/stock_symbols_raw.csv'

# Save the data to a CSV file
all_symbols_df.to_csv(file_path, index=False)

In [9]:
# Out of the all_symbols_df DataFrame, I will create a dictionry with exchange as the key and the symbols as the values (keeping only the symbols with market cap greater than 1 billion)

# Filter the symbols with market cap greater than 50 billion
symbols_filtered_df = all_symbols_df[all_symbols_df['marketCap'] > 50]

# counting the number of symbols by exchange in a table
symbols_count = symbols_filtered_df.groupby('exchange').size().reset_index(name='count')

symbols_count


,exchange,count
0,NASDAQ,272
1,NYSE,321


In [10]:

# Create a dictionary with exchange as the key and the symbols as the values
symbols_50B = symbols_filtered_df.groupby('exchange')['symbol'].apply(list).to_dict()

# Display the symbols for the NASDAQ exchange
symbols_50B['NASDAQ'][:10]

['AAPL',
 'ABALX',
 'ABNB',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'AEGFX',
 'AEP',
 'AEPFX']

In [11]:
# getting a count of unique symbols in the dictionary by exchange

# Count the number of unique symbols for each exchange
symbols_count = {exchange: len(symbols) for exchange, symbols in symbols_50B.items()}
symbols_count

{'NASDAQ': 272, 'NYSE': 321}

In [12]:
# Save the dictionary to a JSON in the data directory to be able to use in the next notebook

# Define the file path
file_path = 'data/Datasets/stock_symbols.json'

# Save the dictionary to a JSON file
import json
with open(file_path, 'w') as f:
    json.dump(symbols_50B, f)
    
symbols_count

{'NASDAQ': 272, 'NYSE': 321}

In [13]:
# As a next step i would like to turn the dictionary into a DataFrame which contains the exchange and the symbols as well as look an get all the company profile information for each symbol. 
# Each of the fields in the company profile will be a column in the DataFrame.
# In order to create this dataset i will create a function that will take in the symbols dictionary and return a DataFrame with the company profile information. The function will through each of the keys in the dictionary and make a request to the API to get the company profile information for each symbol in the list. The function will then return a DataFrame with the company profile information for all the symbols in the dictionary.



# Function to get company profile information for a list of symbol. The API can only do 300 requests a minute so I will add a sleep timer to the function to make sure that the API does not get overloaded.

import time

def get_company_profile(symbols_dict, save_path):
    # Create an empty DataFrame to store the data
    df = pd.DataFrame()

    # Loop through the dictionary
    for exchange, symbols in symbols_dict.items():
        print(f"Fetching company profile information for {exchange} exchange...")
        for symbol in symbols:
            # Define the endpoint
            endpoint = f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}'

            # Make the request
            response = requests.get(endpoint)

            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                # Append the data to the DataFrame
                df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
                # Save the DataFrame to a CSV file incrementally
                df.to_csv(save_path, index=False)
            else:
                print(f"Error: {response.status_code}")

            # Sleep for 0.5 seconds to avoid overloading the API
            time.sleep(0.5)

    return df


In [14]:
# Getting the company profile information for the symbols dictionary

# creating a test dictionary with only 10 symbols for each exchange in the main dictionary named: symbols_10B
test_symbols_dict = {exchange: symbols[:10] for exchange, symbols in symbols_50B.items()}

# Define the file path
file_path_test = 'data/Datasets/company_profile_test.csv'

# Get the company profile information for the test symbols dictionary
company_profile_df = get_company_profile(test_symbols_dict, file_path_test)

# Display the first 5 rows of the DataFrame
company_profile_df

Fetching company profile information for NASDAQ exchange...
Fetching company profile information for NYSE exchange...


,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,AAPL,242.65,1.240000,48361368,3667848870000,0.990,164.08-242.755,3.06,Apple Inc.,USD,...,95014,87.06698,154.358023,https://images.financialmodelingprep.com/symbo...,1980-12-12,False,False,True,False,False
1,ABALX,37.02,0.930000,0,234602938354,0.675,31.01-37.01,0.01,American Funds American Balanced Fund Class A,USD,...,None,-45.26165,78.261652,https://images.financialmodelingprep.com/symbo...,1986-01-02,True,False,True,False,True
2,ABNB,137.46,1.156000,4377857,87177624107,0.000,110.38-170.1,-0.03,"Airbnb, Inc.",USD,...,94103,-9.79184,147.016839,https://images.financialmodelingprep.com/symbo...,2020-12-10,False,False,True,False,False
3,ADBE,516.26,1.299000,2944222,227257652000,0.000,433.97-638.25,0.06,Adobe Inc.,USD,...,95110-2704,55.836,458.584097,https://images.financialmodelingprep.com/symbo...,1986-08-13,False,False,True,False,False
4,ADI,221.54,1.078000,2873546,109949637380,3.680,179.63-244.14,-1.58,"Analog Devices, Inc.",USD,...,01887,-1.81158,224.392682,https://images.financialmodelingprep.com/symbo...,1980-03-17,False,False,True,False,False
5,ADP,303.57,0.795000,1597117,123691721490,6.160,227.12-309.63,-2.46,"Automatic Data Processing, Inc.",USD,...,07068,49.14052,254.659481,https://images.financialmodelingprep.com/symbo...,1980-03-17,False,False,True,False,False
6,ADSK,298.96,1.474000,1428533,64276400000,0.000,195.32-326.62,2.31,"Autodesk, Inc.",USD,...,94903,100.71307,197.730032,https://images.financialmodelingprep.com/symbo...,1985-06-28,False,False,True,False,False
7,AEGFX,58.52,1.050000,0,134359876599,0.921,52.59-60.61,0.45,American Funds EuroPacific Growth Cl F-1 Shs,USD,...,None,None,0.000000,https://images.financialmodelingprep.com/symbo...,2001-03-15,True,False,True,False,True
8,AEP,97.02,0.539000,2814947,51669456300,3.720,75.22-105.18,-1.20,"American Electric Power Company, Inc.",USD,...,43215-2373,84.84514,-0.000141,https://images.financialmodelingprep.com/symbo...,1962-01-02,False,False,True,False,False
9,AEPFX,58.75,1.050000,0,134332758953,1.093,52.71-60.82,0.44,American Funds EuroPacific Growth Fund,USD,...,None,None,0.000000,https://images.financialmodelingprep.com/symbo...,2008-08-01,True,False,True,False,True


In [ ]:
# after successfully getting the company profile information for the test symbols dictionary, I will now get the company profile information for the main dictionary named: symbols_10B

# Define the file path
file_path_main = 'data/Datasets/company_profile_raw.csv'

# Get the company profile information for the symbols dictionary
company_profile_df_main = get_company_profile(symbols_50B, file_path_main)

# Display the first 5 rows of the DataFrame
company_profile_df_main

#saving the company profile data to a csv file

# Define the file path
file_path = 'data/Datasets/company_profile_raw.csv'

# Save the data to a CSV file
company_profile_df_main.to_csv(file_path, index=False)

Fetching company profile information for NASDAQ exchange...


In [ ]:
# Display the first 5 rows of the DataFrame
company_profile_df_main

# make a copy of the company profile data to be able to clean the data and rename the csv file to company_profile_cleaned(50B).csv

# Make a copy of the DataFrame
company_profile_df_cleaned = company_profile_df_main.copy()

# Save the data to a CSV file
f.ile_path = 'data/Datasets/company_profile_cleaned_50B.csv' # Define the file path
company_profile_df_cleaned.to_csv(file_path, index=False) # Save the data to a CSV file